In [16]:
import Pkg; Pkg.add("JSON")

using JSON

   Resolving package versions...


  No Changes to `C:\ENPC\Projects\Kiro\Project.toml`
  No Changes 

to `C:\ENPC\Projects\Kiro\Manifest.toml`
Precompiling 

project...


  ✓ GMP_jll


...

  ✓ Ipopt_jll

  ✓ Clp_jll
  ✓ StaticArrays


  ✓ SCIP_PaPILO_jll


  ✓ SCIP_jll


  ✓ SpecialFunctions


In [ ]:
json_toy = JSON.parsefile("KIRO-tiny.json")


In [ ]:
json_small = JSON.parsefile("KIRO-small.json")

In [ ]:
json_medium = JSON.parsefile("KIRO-medium.json")

In [ ]:
json_large = JSON.parsefile("KIRO-large.json")

In [ ]:
json_huge = JSON.parsefile("KIRO-huge.json")

In [ ]:
# Get Functions

function get_parameters(name_json)
    general_parameters = name_json["general_parameters"]
    return general_parameters
end

function get_wind_turbines(name_json)
    get_wind_turbines = name_json["wind_turbines"]

    return get_wind_turbines
end

function get_wind_scenarios(name_json)
    get_wind_scenarios = name_json["wind_scenarios"]
    return get_wind_scenarios
end

function get_substation_locations(name_json)
    get_substation_locations = name_json["substation_locations"]
    
    return get_substation_locations
end

function get_substation_types(name_json)
    get_substation_types = name_json["substation_types"]

    return get_substation_types
end

function get_land_substation_cable_types(name_json)
    get_land_substation_cable_types = name_json["land_substation_cable_types"]

    return get_land_substation_cable_types
end

function get_substation_substation_cable_types(name_json)
    get_substation_substation_cable_types = name_json["substation_substation_cable_types"]

    return get_substation_substation_cable_types
end

In [ ]:
# Fonctions coûts 

function cost_turbines_substation(L, D1, general_parameters)
    # L : vecteur liant turbines et sous-stations
    # D1 : matrice des distances entre les turbines et les sous-stations
    # general_parameters : paramètres généraux du problème

    # On initialise le coût
    cost = 0

    # On calcule le coût de la liaison entre les turbines et les sous-stations
    for i in 1:length(L)
        cost += general_parameters["C_ft"] +D1[i,L[i]]*general_parameters["C_lt"]
    end

    return cost
end

function cost_substation_substation(S, D2, substation_substation_cable_types)
    # S : vecteur liant les sous-stations
    # D2 : matrice des distances entre les sous-stations
    # substation_substation_cable_types : types de câbles entre les sous-stations

    # On initialise le coût
    cost = 0

    # On calcule le coût de la liaison entre les sous-stations
    for i in 1:length(S)
        cost += substation_substation_cable_types[S[i][3]]["fixed_cost"]+D2[i,S[i][2]]*substation_substation_cable_types[S[i][3]]["variable_cost"]
    end

    cost = cost/2

    return cost
end

function cost_substation_land_substation(S, D3, land_substation_cable_types)
    # S : vecteur liant les sous-stations
    # D3 : vecteur des distances entre les sous-stations et la station terrestre
    # land_substation_cable_types : types de câbles entre les sous-stations et la station terrestre

    # On initialise le coût
    cost = 0

    # On calcule le coût de la liaison entre les sous-stations et la station terrestre
    for i in 1:length(S)
        cost += land_substation_cable_types[S[i][4]]["fixed_cost"]+D3[i]*land_substation_cable_types[S[i][4]]["variable_cost"]
    end
end

function cost_substation(S, substation_types)
    # S : vecteur liant les sous-stations
    # substation_types : types de sous-stations

    # On initialise le coût
    cost = 0

    # On calcule le coût de construction des sous-stations
    for i in 1:length(S)
        cost += substation_types[S[i][1]]["cost"]
    end

    return cost
end


function c_c(C, general_parameters)

    return general_parameters["curtailing_cost"]*C + general_parameters["curtailing_penalty"]*max(0,C-general_parameters["maximum_curtailing"])
end


function C_f_aux(v, v1, w, wind_scenarios, S, substation_substation_cable_types, land_substation_cable_types, substation_types)
    C_f_aux = 0

    C_f_aux += wind_scenarios[w][" power_generation "]*S[v1][1]

    if S[v1][2]==v
        C_f_aux += min(substation_substation_cable_types[S[v1][3]]["rating"], wind_scenarios[w][" power_generation "]*S[v][1])
    else
        C_f_aux += min(0, wind_scenarios[w][" power_generation "]*S[v][1])
    end

    if S[v1][4]!=0
        C_f_aux -= min(substation_types[S[v1][5]],land_substation_cable_types[S[v1][4]]["rating"])
    end
end


function C_f(v, w, wind_scenarios, S, substation_substation_cable_types, land_substation_cable_types, substation_types)
    C_f = 0

    if S[v][2]!=0
        C_f += max(0, wind_scenarios[w][" power_generation "]*S[v][1]-substation_substation_cable_types[S[v][3]]["rating"])
    else
        C_f += max(0, wind_scenarios[w][" power_generation "]*S[v][1])
    end

    for i in 1:length(S)
        if S[i][4]!=0
            C_f += max(0, C_f_aux(v, i, w, wind_scenarios, S, substation_substation_cable_types, land_substation_cable_types, substation_types))
        end
    end

    return C_f

end


function C_n(w, wind_scenarios, S, substation_substation_cable_types, land_substation_cable_types, substation_types)
    
    C_n = 0

    for i in 1:length(S)
        if S[i][4] != 0
            C_n += max(0, wind_scenarios[w][" power_generation "]*S[i][1]-min(substation_types[S[i][5]],land_substation_cable_types[S[i][4]]["rating"]))
        end
    end

    return C_n
end
    

function p_f(v, S, land_substation_cable_types, substation_types)
    p_f = 0

    p_f += land_substation_cable_types[S[v][4]]["probability_of_failure "]

    p_f += substation_types[S[v][5]]["probability_of_failure "]

    return p_f
end


function operational_cost_aux(S, w, wind_scenarios, general_parameters, substation_substation_cable_types, land_substation_cable_types, substation_types)
    cost1 = 0
    cost2 = 0

    for i in 1:length(S)
        cost1 += p_f(i, S, land_substation_cable_types, substation_types)*c_c(C_f(i, w, wind_scenarios, S, substation_substation_cable_types, land_substation_cable_types, substation_types,general_parameters))
    end

    for i in 1:length(S)
        cost2 += p_f(i, S, land_substation_cable_types, substation_types)
    end
    cost2 = (1 - cost2)*c_c(C_n(w, wind_scenarios, S, substation_substation_cable_types, land_substation_cable_types, substation_types,general_parameters))

    return cost1 + cost2
end

function operational_cost(L, S, wind_scenarios, general_parameters, substation_substation_cable_types, land_substation_cable_types, substation_types)
    # L : vecteur liant turbines et sous-stations
    # S : vecteur liant les sous-stations
    # wind_scenarios : scénarios de vent
    # general_parameters : paramètres généraux du problème

    # On initialise le coût
    cost = 0

    for w in 1:length(wind_scenarios)
        cost += wind_scenarios[w]["probability"]*operational_cost_aux(S, w, wind_scenarios, general_parameters, substation_substation_cable_types, land_substation_cable_types, substation_types)
        
    end

    return cost
end

function cost(L, S, D1, D2, D3, wind_scenarios, general_parameters, substation_substation_cable_types, land_substation_cable_types, substation_types)
    # L : vecteur liant turbines et sous-stations
    # S : vecteur liant les sous-stations
    # D1 : matrice des distances entre les turbines et les sous-stations
    # D2 : matrice des distances entre les sous-stations
    # D3 : vecteur des distances entre les sous-stations et la station terrestre
    # wind_scenarios : scénarios de vent
    # general_parameters : paramètres généraux du problème
    # substation_substation_cable_types : types de câbles entre les sous-stations
    # land_substation_cable_types : types de câbles entre les sous-stations et la station terrestre
    # substation_types : types de sous-stations

    # On initialise le coût
    cost = 0

    # On calcule le coût de la liaison entre les turbines et les sous-stations
    cost += cost_turbines_substation(L, D1, general_parameters)

    # On calcule le coût de la liaison entre les sous-stations
    cost += cost_substation_substation(S, D2, substation_substation_cable_types)

    # On calcule le coût de la liaison entre les sous-stations et la station terrestre
    cost += cost_substation_land_substation(S, D3, land_substation_cable_types)

    # On calcule le coût de construction des sous-stations
    cost += cost_substation(S, substation_types)

    # On calcule le coût opérationnel
    cost += operational_cost(L, S, wind_scenarios, general_parameters, substation_substation_cable_types, land_substation_cable_types, substation_types)

    return cost
end

In [ ]:
# Heuristic Functions

function heuristique_1(D1, D2)
    # D1 : matrice des distances entre les turbines et les sous-stations
    # D2 : matrice des distances entre les sous-stations

    # On initialise le vecteur liant turbines et sous-stations
    L = zeros(length(D1),1)

    # On initialise le vecteur des sous-stations
    S = zeros(length(D2),1)

    # On emplit ce vecteur en associant à chaque turbine la sous-station la plus proche
    for i in 1:length(D1)
        L[i] = argmin(D1[i,:])
        S[L[i]][1] += 1
    end

    # On emplit ce vecteur en associant à chaque sous-station présente dans L la sous-station la plus proche
    for i in 1:length(L)
        if S[L[i]][2] == 0
            S[L[i]][2] = argmin(D2[L[i],:])
            S[S[L[i]][2]][2] = L[i]
        end
    end

    return L,S
end

In [ ]:
#  Création des matrices de distances des turbines aux sous-stations, des sous-stations à la station terrestre et des sous-stations entre elles

function distance(V_t,V_s,v_0)
    # V_t : vecteur des turbines
    # V_s : vecteur des sous-stations
    # v_0 : vecteur de la station terrestre
    
    # On initialise le vecteur des distances
    D1 = zeros(length(V_t),length(V_s))
    D2 = zeros(length(V_s),length(V_s))
    D3 = zeros(length(V_s),1)

    # On calcule les distances entre les turbines et les sous-stations, et entre les sous-stations et la station terrestre et entre les sous-stations
    for i in 1:length(V_t)
        for j in 1:length(V_s)
            D1[i,j] = sqrt((V_t[i][1]-V_s[j][1])^2 + (V_t[i][2]-V_s[j][2])^2)
        end
    end

    for i in 1:length(V_s)
        for j in 1:length(V_s)
            D2[i,j] = sqrt((V_s[i][1]-V_s[j][1])^2 + (V_s[i][2]-V_s[j][2])^2)
        end
        D2[i,i] = +Inf
    end

    for i in 1:length(V_s)
        D3[i] = sqrt((V_s[i][1]-v_0[1])^2 + (V_s[i][2]-v_0[2])^2)
    end

    return D1,D2,D3
end

In [ ]:
# Heuristique ; on lie les turbines aux sous-stations les plus proches

function heuristique_1(D1, D2, substation_substation_cable_types, land_substation_cable_types, substation_types)
    # D1 : matrice des distances entre les turbines et les sous-stations
    # D2 : matrice des distances entre les sous-stations
    # D3 : vecteur des distances entre les sous-stations et la station terrestre

    # On choisit aléatoirement une turbine, notée n, on la lie avec la sous-station la plus proche de celle à laquelle est elle actuellement associée
    L1 = L
    S1 = S

    new = rand(1:length(L))

    # On emplit ce vecteur en associant à chaque turbine la sous-station la plus proche
    for i in 1:length(D1)
        L[i] = argmin(D1[i,:])
        S[L[i]][5] = substation_types[-1]
        S[L[i]][3] = land_substation_cable_types[-1]
        S[L[i]][1] += 1
    end

    # On emplit ce vecteur en associant à chaque sous-station présente dans L la sous-station présente dans L la plus proche
    for i in 1:length(L)
        if S[L[i]][2] == 0 && S[argmin(D2[L[i],L])][2] == 0
            S[L[i]][2] = argmin(D2[L[i],L])
            S[S[L[i]][2]][2] = L[i]
            S[L[i]][4] = substation_substation_cable_types[-1]
            S[S[L[i]][2]][4] = substation_substation_cable_types[-1]
        end
    end

    if cost(L1, S1, D1, D2, D3) < cost(L, S, D1, D2, D3)
        return L1, S1
    else
        return L, S
    end
end

function voisinage_substation_aux_1(L, S, D1, D2, D3)
    # L : vecteur liant turbines et sous-stations
    # S : vecteur liant les sous-stations
    # D1 : matrice des distances entre les turbines et les sous-stations
    # D2 : matrice des distances entre les sous-stations
    # D3 : vecteur des distances entre les sous-stations et la station terrestre

    # On choisit aléatoirement une sous-station reliée à une turbine, notée n, on la lie avec la sous-station la plus proche de celle à laquelle est elle actuellement associée
    L1 = L
    S1 = S

    new = rand(1:length(L))

    new = L1[new]

    old = S1[new][2]
    S1[old][2] = 0

    if S1[argmin(D2[old,:])][2]==0 && S1[argmin(D2[old,:])][1]!=0
        S1[argmin(D2[old,:])][2] = new
        S1[new][2] = argmin(D2[old,:])
    else
        S1[new][2]=0
    end
    
end

In [ ]:
# Fonctions voisinages
# Ces fonctions sont la base de la méta-heuristique ; elles effectuent les changements de lien entre turbines et sous-stations et entre sous-stations entre elles

function voisinage_turbine_substation(L, S, D1, D2, D3, wind_scenarios, general_parameters, substation_substation_cable_types, land_substation_cable_types, substation_types)
    # L : vecteur liant turbines et sous-stations
    # S : vecteur liant les sous-stations
    # D1 : matrice des distances entre les turbines et les sous-stations
    # D2 : matrice des distances entre les sous-stations
    # D3 : vecteur des distances entre les sous-stations et la station terrestre

    # On choisit aléatoirement une turbine, notée n, on la lie avec la sous-station la plus proche de celle à laquelle est elle actuellement associée
    c = cost(L, S, D1, D2, D3, wind_scenarios, general_parameters, substation_substation_cable_types, land_substation_cable_types, substation_types)

    new = rand(1:length(L))

    old = L[new]
    linked_old = S[old][2]

    # On change la liaison entre la turbine et la sous-station en choisissant la sous-station la plus proche de l'ancienne sous-station
    L[new] = argmin(D2[old,filter(x -> x != new, L)])
    S[old][1] -= 1

    # Si la sous-station n'est plus reliée à aucune turbine, on la supprime
    if S[old][1] == 0
        S[linked_old][2] = 0
        S[linked_old][4] = 0
        S[old][2] = 0
        S[old][3] = 0
        S[old][4] = 0
        S[old][5] = 0
    end

    S[L[new]][1] += 1
    if linked_old != 0 && S[L[new]][2] == 0
        S[L[new]][2] = linked_old
        S[linked_old][2] = L[new]
        
    end

    if cost(L, S, D1, D2, D3, wind_scenarios, general_parameters, substation_substation_cable_types, land_substation_cable_types, substation_types) < c
        return L, S
    else
        return None
    end
end

function voisinage_substation_substation(L, S, D1, D2, D3, wind_scenarios, general_parameters, substation_substation_cable_types, land_substation_cable_types, substation_types)
    # L : vecteur liant turbines et sous-stations
    # S : vecteur liant les sous-stations
    # D1 : matrice des distances entre les turbines et les sous-stations
    # D2 : matrice des distances entre les sous-stations
    # D3 : vecteur des distances entre les sous-stations et la station terrestre

    # On choisit aléatoirement une sous-station, notée n, on la lie avec la sous-station la plus proche de celle à laquelle est elle actuellement associée
    c = cost(L, S, D1, D2, D3, wind_scenarios, general_parameters, substation_substation_cable_types, land_substation_cable_types, substation_types)
    L1 = L
    S1 = S
    substation = rand(1:length(L))

    old = S[substation][2]

    # On change la liaison entre la sous-station et la sous-station en choisissant la sous-station liée à une turbine la plus proche de l'ancienne sous-station
    new = L[1][argmin(D2[old,filter(x -> x != substation, L)])]
    S[old][2] = S[new][2]
    S[old[2]][2] = old
    S[new][2] = substation
    S[substation][2] = new

    S[old][4] = substation_substation_cable_types[-1]
    S[new][4] = substation_substation_cable_types[-1]
    S[substation][4] = substation_substation_cable_types[-1]
    S[old[2]][4] = substation_substation_cable_types[-1]

    if cost(L, S, D1, D2, D3, wind_scenarios, general_parameters, substation_substation_cable_types, land_substation_cable_types, substation_types) < c
        return L, S
    else
        return L1, S1
    end
end


In [ ]:
# Métaheuristique

function metaheuristique(wind_scenarios, general_parameters, substation_substation_cable_types, land_substation_cable_types, substation_types)
    D1, D2, D3 = distance(V_t,V_s,coords_mls)
    L, S = heuristique_1(D1, D2, substation_substation_cable_types, land_substation_cable_types, substation_types)
    costs = Float64[]
    for i in 1000
        if rand() < 0.5
            L, S = voisinage_turbine_substation(L, S, D1, D2, D3, wind_scenarios, general_parameters, substation_substation_cable_types, land_substation_cable_types, substation_types)
        else
            L, S = voisinage_substation_substation(L, S, D1, D2, D3, wind_scenarios, general_parameters, substation_substation_cable_types, land_substation_cable_types, substation_types)
        end 
        if i/10 == 0
            new_cost = cost(L, S, D1, D2, D3, wind_scenarios, general_parameters, substation_substation_cable_types, land_substation_cable_types, substation_types)
            push!(costs, new_cost)
        end
    end
    final_cost = cost(L, S, D1, D2, D3, wind_scenarios, general_parameters, substation_substation_cable_types, land_substation_cable_types, substation_types)
    plot(1:1000, costs, xlabel = "Itérations", ylabel = "Coût", title = "Évolution du coût")
    return L, S, final_cost
end

In [ ]:
#collectes des données utiles

file = json_small

P_max = get_parameters(file)["maximum_power"]
coords_mls = get_parameters(file)["main_land_station"]
C_ft = get_parameters(file)["fixed_cost_cable"]
C_lt = get_parameters(file)["variable_cost_cable"]
C_0 = get_parameters(file)["curtailing_cost"]
C_p = get_parameters(file)["curtailing_penalty"]
C_max = get_parameters(file)["maximum_curtailing"]
general_parameters = Dict("maximum_power" => P_max, "main_land_station" => coords_mls, "fixed_cost_cable" => C_ft, "variable_cost_cable" => C_lt, "curtailing_cost" => C_0, "curtailing_penalty" => C_p, "maximum_curtailing" => C_max)

wind_turbines = get_wind_turbines(file)
x_turbines = [turbine["x"] for turbine in wind_turbines]
y_turbines = [turbine["y"] for turbine in wind_turbines]
V_t = [x_turbines y_turbines]

wind_scenarios = get_wind_scenarios(file)

substation_locations = get_substation_locations(file)
x_substations = [substation["x"] for substation in substation_locations]
y_substations = [substation["y"] for substation in substation_locations]
V_s = [x_substations y_substations]

substation_types = get_substation_types(file)

land_substation_cable_types = get_land_substation_cable_types(file)

sub_sub_cable_types = get_substation_substation_cable_types(file)


In [ ]:
# Execution du programme

L, S, cost = metaheuristique(wind_scenarios, general_parameters, sub_sub_cable_types, land_substation_cable_types, substation_types)

In [ ]:
# Formatage en Json